In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import pandas as pd
import seaborn as sns
import scipy

import pymc3 as pm
import theano
import theano.tensor as tt
from theano import tensor as T
import arviz as az

from NUTS_more_pars_helpers import generate_data, llik_td_vectorized#, update_Q, theano_llik_td, get_nuts_est

## Parameter recoverability using NUTS with more pars

In [2]:
true_alpha_neg = .3
true_alpha_pos = .3
true_exp_neg = 1
true_exp_pos = 1
true_beta = 1.5
#est_df = get_nuts_est(t_alpha_neg = true_alpha_neg, t_alpha_pos = true_alpha_pos, t_exp_neg = true_exp_neg, t_exp_pos = true_exp_pos, t_beta = true_beta, n=60)

In [3]:
machines, actions, rewards, all_Qs = generate_data(alpha_neg= true_alpha_neg, alpha_pos= true_alpha_pos, exp_neg= true_exp_neg, exp_pos= true_exp_pos, beta= true_beta)

In [4]:
llik_td_vectorized([true_alpha_neg, true_alpha_pos, true_exp_neg, true_exp_pos, true_beta], *(machines, actions, rewards))

12.096083576469178

In [5]:
machines_ = theano.shared(np.asarray(machines, dtype='int16'))
actions_ = theano.shared(np.asarray(actions, dtype='int16'))
rewards_ = theano.shared(np.asarray(rewards, dtype='int16'))

In [6]:
Qs = tt.zeros((4,2), dtype='float64')

In [7]:
def update_Q(machine, action, reward,
             Q,
            alpha_neg, alpha_pos, exp_neg, exp_pos):

    rpe = reward- Q[machine, action]
    
    Q_upd = tt.switch(rpe<0, Q[machine, action] + alpha_neg * np.sqrt((rpe)**2)**exp_neg * (-1), Q[machine, action] + alpha_pos * rpe**exp_pos)
    
    Q = tt.set_subtensor(Q[machine, action], Q_upd)
    return Q

In [8]:
tmp = update_Q(machines_[0], actions_[0], rewards_[0], Qs, true_alpha_neg, true_alpha_pos, true_exp_neg, true_exp_pos)
tmp.eval()

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., 30.]])

In [9]:
Qs = tt.zeros((4,2), dtype='float64')
alpha_neg = tt.scalar("alpha_neg")
alpha_pos = tt.scalar("alpha_pos")
exp_neg = tt.scalar("exp_neg")
exp_pos = tt.scalar("exp_pos")
Qs, updates = theano.scan(
        fn=update_Q,
        sequences=[machines_, actions_, rewards_],
        outputs_info=[Qs],
        non_sequences=[alpha_neg, alpha_pos, exp_neg, exp_pos],
        strict=True)

theano_llik_td = theano.function(inputs=[alpha_neg, alpha_pos, exp_neg, exp_pos], outputs=[Qs], updates = updates)
result = theano_llik_td(true_alpha_neg, true_alpha_pos, true_exp_neg, true_exp_pos)

In [10]:
with pm.Model() as m:
    s_alpha_neg = pm.Beta('alpha_neg', 1, 1)
    s_alpha_pos = pm.Beta('alpha_pos', 1, 1)
    s_exp_neg = pm.Beta('exp_neg', 1, 1)
    s_exp_pos = pm.Beta('exp_pos', 1, 1)
    s_beta = pm.HalfNormal('beta', 10)

    like = pm.Potential('like', theano_llik_td(s_alpha_neg, s_alpha_pos, s_exp_neg, s_exp_pos, s_beta, machines, actions, rewards, n=100))
    tr = pm.sample()

[array([[[   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,   30.        ]],
 
        [[   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,   30.        ]],
 
        [[   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,   51.        ]],
 
        [[   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,   51.        ]],
 
        [[   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,   51.        ]],
 
        [[   0.        ,    0.        ],
         [   0.        ,    0.        ],
         [   0.        ,  -30.        ],
         [   0.        ,   51.        ]],


Next steps:

Simulations:

-  Can pyMC recover parameters better for the whole range?
    -  Conclusion: Not necessarily but   
        - it keeps all values within bounds
        - it does not fail in any other way compared to the realistic MLE estimates (i.e. those starting from random values instead of the true values used for data generation)  
        
- Is recoverability worse for generating processes with more parameters
    - TODO: Write new/more generalizable ~~data generation~~ and par recovery functions for different RL models (with more parameters)
    - Compare parameter recoverability for at least one more RL model with more parameters 
    - Not interested in comparing MLE versus NUTS but compare simulation results of NUTS estimates between more simple and complicated processes
    
Fitting:

- Convert the recoverability function to fitting function

```
 # NUTS estimate
    actions_ = theano.shared(np.asarray(actions, dtype='int16'))
    with pm.Model() as m:
        alpha = pm.Beta('alpha', 1, 1)
        beta = pm.HalfNormal('beta', 10)
        like = pm.Potential('like', theano_llik_td(alpha, beta, machines, actions, rewards, n))
        tr = pm.sample()
    
    nuts_alpha_ave = np.mean(tr.alpha)
    nuts_beta_ave = np.mean(tr.beta)
    nuts_alpha_std = np.std(tr.alpha)
    nuts_beta_std = np.std(tr.beta)
    nuts_llik = llik_td_vectorized([nuts_alpha_ave, nuts_beta_ave], *(machines, actions, rewards))
```
- Fit to one subject's data (using the fitting function created above in fit_rl_cv.py)
- Fit to all subjects' data